In [1]:
import os.path
import random
import time
from functools import partial
from tkinter import *
import sys

from lib.EightPuzzle import EightPuzzle

In [2]:
root = Tk()

state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
puzzle = EightPuzzle(tuple(state))
solution = None

b = [None] * 9

In [3]:
def manathanhuristic(start, puzzle):
    x = puzzle[start]
    s_x = 0
    s_y = 0
    p_x = 0
    p_y = 0
    if start <= 2:
        s_y = 0
        s_x = start
    elif start > 2 and start <= 5:
        s_y = 1
        s_x = start - 3
    elif start > 5 and start <= 8:
        s_y = 2
        s_x = start - 6

    if x == 0:
        p_y = 2
        p_x = 2
    elif x > 0 and x <= 3:
        p_y = 0
        p_x = x - 1
    elif x > 3 and x <= 6:
        p_y = 1
        p_x = x - 4
    elif x > 6 and x <= 8:
        p_y = 2
        p_x = x - 7

    return abs(s_x - p_x) + abs(s_y - p_y)


def misplacedhuristic(puzzle):
    count = 0
    for i in range(8):
        if (i + 1) != puzzle[i]:
            count += 1
    if puzzle[8] != 0:
        count += 1
    return count


def checkgoalstate(p1, path1, index1):
    path=path1.copy()
    index=index1
    p=p1
    path.pop(0)

    for i in range(len(path)):
        if path[i] == 'LEFT':
            p[index] = p[index - 1]
            p[index - 1] = 0
            index=index-1
        elif path[i] == 'RIGHT':
            p[index] = p[index + 1]
            p[index + 1] = 0
            index=index+1
        elif path[i] == 'UP':
            p[index] = p[index - 3]
            p[index - 3] = 0
            index=index-3
        elif path[i] == 'DOWN':
            p[index] = p[index + 3]
            p[index + 3] = 0
            index=index+3
    x = misplacedhuristic(p)
    if x == 0:
        return True,p
    else:
        return False,p


def husristic(start, puzzle):
    distance = manathanhuristic(start,puzzle)
    tiles = misplacedhuristic(puzzle)
    return max(distance, tiles)


def solve():
    """
    Solves the puzzle using astar_search using puzzle array
    p contains the input puzzle
    goal

    return an array of this structure
    ['UP', 'LEFT', 'UP', 'RIGHT', 'DOWN', 'LEFT', 'LEFT', 'DOWN', 'RIGHT', 'DOWN']
    """
    global puzzle
    p=puzzle.initial
    x=list(p)
    
    goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]
    timecomplexity = 1
    spacecomplexity = 1
    # implement the algorithm here
    p_0 = 0
    for i in range(9):
        if p[i] == 0:
            p_0 = i
            break
    c_h = husristic(p_0, x.copy())
    queue = [(p_0, ['start'], c_h)]
    visited = []
    while len(queue) > 0:
        queue = sorted(queue, key=lambda x: x[2])
        spacecomplexity = max(len(queue), spacecomplexity)
        c_index, path, hur = queue[0]
        queue.pop(0)
        timecomplexity += 1
        check,npath = checkgoalstate(x.copy(), path.copy(), p_0)
        if check == True:
            path.pop(0)
            return path,timecomplexity,spacecomplexity
        if c_index != 0 and c_index != 3 and c_index != 6 and (npath) not in visited:
            temp = path + ['LEFT']
            c_h = husristic(c_index - 1, npath)
            queue.append((c_index - 1, temp, c_h))
        if c_index != 0 and c_index != 1 and c_index != 2 and (npath) not in visited:
            temp = path + ['UP']
            c_h = husristic(c_index - 3, npath)
            queue.append((c_index - 3, temp, c_h))
        if c_index != 2 and c_index != 5 and c_index != 8 and (npath) not in visited:
            temp = path + ['RIGHT']
            c_h = husristic(c_index + 1, npath)
            queue.append((c_index + 1, temp, c_h))
        if c_index != 6 and c_index != 7 and c_index != 8 and (npath) not in visited:
            temp = path + ['DOWN']
            c_h = husristic(c_index + 3, npath)
            queue.append((c_index + 3, temp, c_h))

        if npath not in visited:
            visited.append(npath)
    
    return [],timecomplexity,spacecomplexity

In [4]:


def scramble():
    """Scrambles the puzzle starting from the goal state"""

    global state
    global puzzle
    possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
    scramble = []
    for _ in range(60):
        scramble.append(random.choice(possible_actions))

    for move in scramble:
        if move in puzzle.actions(state):
            state = list(puzzle.result(state, move))
            puzzle = EightPuzzle(tuple(state))
            create_buttons()


def solve_steps():
    """Solves the puzzle step by step"""

    global puzzle
    global solution
    global state
    solution,timecomplexity,spacecomplexity = solve()    
    print(solution)
    print(f'Time Complexity: {timecomplexity}')
    print(f'Space Complexity: {spacecomplexity}')

    for move in solution:
        state = puzzle.result(state, move)
        create_buttons()
        root.update()
        root.after(1, time.sleep(0.75))


def exchange(index):
    """Interchanges the position of the selected tile with the zero tile under certain conditions"""

    global state
    global solution
    global puzzle
    zero_ix = list(state).index(0)
    actions = puzzle.actions(state)
    current_action = ''
    i_diff = index // 3 - zero_ix // 3
    j_diff = index % 3 - zero_ix % 3
    if i_diff == 1:
        current_action += 'DOWN'
    elif i_diff == -1:
        current_action += 'UP'

    if j_diff == 1:
        current_action += 'RIGHT'
    elif j_diff == -1:
        current_action += 'LEFT'

    if abs(i_diff) + abs(j_diff) != 1:
        current_action = ''

    if current_action in actions:
        b[zero_ix].grid_forget()
        b[zero_ix] = Button(root, text=f'{state[index]}', width=6, font=('Helvetica', 40, 'bold'),
                            command=partial(exchange, zero_ix))
        b[zero_ix].grid(row=zero_ix // 3, column=zero_ix % 3, ipady=40)
        b[index].grid_forget()
        b[index] = Button(root, text=None, width=6, font=('Helvetica', 40, 'bold'), command=partial(exchange, index))
        b[index].grid(row=index // 3, column=index % 3, ipady=40)
        state[zero_ix], state[index] = state[index], state[zero_ix]
        puzzle = EightPuzzle(tuple(state))


def create_buttons():
    """Creates dynamic buttons"""

    # TODO: Find a way to use grid_forget() with a for loop for initialization
    b[0] = Button(root, text=f'{state[0]}' if state[0] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 0))
    b[0].grid(row=0, column=0, ipady=40)
    b[1] = Button(root, text=f'{state[1]}' if state[1] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 1))
    b[1].grid(row=0, column=1, ipady=40)
    b[2] = Button(root, text=f'{state[2]}' if state[2] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 2))
    b[2].grid(row=0, column=2, ipady=40)
    b[3] = Button(root, text=f'{state[3]}' if state[3] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 3))
    b[3].grid(row=1, column=0, ipady=40)
    b[4] = Button(root, text=f'{state[4]}' if state[4] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 4))
    b[4].grid(row=1, column=1, ipady=40)
    b[5] = Button(root, text=f'{state[5]}' if state[5] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 5))
    b[5].grid(row=1, column=2, ipady=40)
    b[6] = Button(root, text=f'{state[6]}' if state[6] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 6))
    b[6].grid(row=2, column=0, ipady=40)
    b[7] = Button(root, text=f'{state[7]}' if state[7] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 7))
    b[7].grid(row=2, column=1, ipady=40)
    b[8] = Button(root, text=f'{state[8]}' if state[8] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
                  command=partial(exchange, 8))
    b[8].grid(row=2, column=2, ipady=40)


def create_static_buttons():
    """Creates scramble and solve buttons"""

    scramble_btn = Button(root, text='Scramble', font=('Helvetica', 30, 'bold'), width=8, command=partial(init))
    scramble_btn.grid(row=3, column=0, ipady=10)
    solve_btn = Button(root, text='Solve', font=('Helvetica', 30, 'bold'), width=8, command=partial(solve_steps))
    solve_btn.grid(row=3, column=2, ipady=10)


def init():
    """Calls necessary functions"""

    global state
    global solution
    state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
    scramble()
    create_buttons()
    create_static_buttons()


init()
root.mainloop()


['LEFT', 'DOWN', 'RIGHT', 'DOWN', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'UP', 'RIGHT', 'DOWN', 'RIGHT', 'UP', 'LEFT', 'LEFT', 'DOWN', 'DOWN', 'RIGHT', 'RIGHT', 'UP', 'LEFT', 'DOWN', 'LEFT', 'UP', 'RIGHT', 'RIGHT', 'DOWN']
Time Complexity: 1947
Space Complexity: 1254
['DOWN', 'LEFT', 'DOWN', 'RIGHT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'UP', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'UP', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'UP', 'UP', 'RIGHT', 'DOWN', 'LEFT', 'UP', 'RIGHT', 'DOWN', 'DOWN', 'LEFT', 'UP', 'RIGHT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'DOWN', 'LEFT', 'UP', 'UP', 'RIGHT', 'DOWN', 'LEFT', 'DOWN', 'RIGHT', 'UP', 'UP', 'RIGHT', 'DOWN', 'DOWN', 'LEFT', 'UP', 'RIGHT', 'UP', 'LEFT', 'DOWN', 'DOWN', 'RIGHT']
Time Complexity: 2131
Space Complexity: 1358


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\abdul\AppData\Local\Temp\ipykernel_13396\173419860.py", line 31, in solve_steps
    create_buttons()
  File "C:\Users\abdul\AppData\Local\Temp\ipykernel_13396\173419860.py", line 76, in create_buttons
    b[0] = Button(root, text=f'{state[0]}' if state[0] != 0 else None, width=6, font=('Helvetica', 40, 'bold'),
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 2726, in __init__
    Widget.__init__(self, master, 'button', cnf, kw)
  File "c:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 2648, in __init__
    self.tk.call(
_tkinter.TclError: can't invok